In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [15]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/졸업논문/data/template_merged_output.csv')
df['predicate'].fillna('Not Dark Pattern', inplace=True)
df['predicate'].unique()

/tmp/ipython-input-3255758935.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['predicate'].fillna('Not Dark Pattern', inplace=True)


array(['Not Dark Pattern', 'Activity Notifications', 'Countdown Timers',
       'Limited-time Messages', 'Low-stock Messages', 'Confirmshaming',
       'Pressured Selling', 'High-demand Messages', 'Trick Questions',
       'Testimonials of Uncertain Origin'], dtype=object)

In [16]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from torch.optim import AdamW
import torch.nn as nn
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

# GPU 사용 설정 (가능한 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 1. 데이터 준비 📝
# ------------------------------------

# 각 문자열 레이블을 리스트로 감싸줍니다.
target_labels = df['predicate'].apply(lambda x: [x])

# MultiLabelBinarizer를 사용하여 'predicate' 레이블을 이진 벡터로 변환
mlb = MultiLabelBinarizer()
encoded_labels = mlb.fit_transform(target_labels)
num_labels = len(mlb.classes_)
print(f"총 Predicate 레이블 수: {num_labels}")
print(f"Predicate 종류: {mlb.classes_}")

# 데이터셋 분리 (학습용, 검증용)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['String'].tolist(),
    encoded_labels,
    test_size=0.2,
    random_state=42
)

# BERT 토크나이저 로딩
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 커스텀 데이터셋 클래스 (변경 없음)
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = torch.FloatTensor(self.labels[idx])

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': label
        }

# 데이터셋 및 데이터로더 생성
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)


# 2. 모델 아키텍처 정의 🏗️
# ------------------------------------

# 모델 클래스 (변경 없음)
class BERTMultiLabelClassifier(nn.Module):
    def __init__(self, bert_model_name, num_labels):
        super(BERTMultiLabelClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# ✨ 변경점: num_labels가 predicate의 개수로 자동 설정됩니다.
model = BERTMultiLabelClassifier('bert-base-uncased', num_labels)
model.to(device)


# 3. 모델 학습 및 최적화 🚀
# ------------------------------------

# 학습 로직 (변경 없음)
criterion = nn.BCEWithLogitsLoss()
optimizer = AdamW(model.parameters(), lr=3e-5)

epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} | Training Loss: {avg_train_loss:.4f}")

print("\n🎉 학습 완료!")

# 4. 평가 및 추론 📊
# ------------------------------------

# 예측 함수 (변경 없음)
def predict(text):
    model.eval()

    encoding = tokenizer.encode_plus(
        text, add_special_tokens=True, max_length=128, return_token_type_ids=False,
        padding='max_length', truncation=True, return_attention_mask=True, return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.sigmoid(outputs)

    preds_indices = (probs > 0.5).cpu().numpy()[0].nonzero()[0]
    predicted_labels = mlb.classes_[preds_indices]

    return predicted_labels.tolist()


Using device: cuda
총 Predicate 레이블 수: 10
Predicate 종류: ['Activity Notifications' 'Confirmshaming' 'Countdown Timers'
 'High-demand Messages' 'Limited-time Messages' 'Low-stock Messages'
 'Not Dark Pattern' 'Pressured Selling' 'Testimonials of Uncertain Origin'
 'Trick Questions']
Epoch 1/10 | Training Loss: 0.1475
Epoch 2/10 | Training Loss: 0.0420
Epoch 3/10 | Training Loss: 0.0236
Epoch 4/10 | Training Loss: 0.0181
Epoch 5/10 | Training Loss: 0.0108
Epoch 6/10 | Training Loss: 0.0083
Epoch 7/10 | Training Loss: 0.0041
Epoch 8/10 | Training Loss: 0.0049
Epoch 9/10 | Training Loss: 0.0129
Epoch 10/10 | Training Loss: 0.0085

🎉 학습 완료!


'\n# 새로운 텍스트로 예측 실행\nnew_text_1 = "Google unveiled a new AI model for its search engine."\npredictions_1 = predict(new_text_1)\nprint(f"\n입력 텍스트: \'{new_text_1}\'")\nprint(f"예측된 Predicate: {predictions_1}") # ✨ 출력 메시지 변경\n\nnew_text_2 = "A lion is chasing a gazelle."\npredictions_2 = predict(new_text_2)\nprint(f"\n입력 텍스트: \'{new_text_2}\'")\nprint(f"예측된 Predicate: {predictions_2}") # ✨ 출력 메시지 변경\n'

# Not Dark Pattern이여야 함

In [22]:
# 1. 여러 문장을 한 번에 예측하는 함수 정의
def predict_batch(texts: list):
    model.eval()  # 모델을 평가 모드로 설정

    # ✨ 변경점 1: 토크나이저에 문장 리스트를 그대로 전달하고, padding=True 옵션 추가
    encoding = tokenizer(
        texts,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding=True,  # 배치 내 문장 길이를 맞춰줌
        truncation=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.sigmoid(outputs)

    # ✨ 변경점 2: 배치 전체의 결과를 한 번에 처리
    # 임계값(0.5)을 기준으로 예측 결정
    preds = (probs > 0.5).cpu().numpy()

    all_predicted_labels = []
    # 각 문장의 예측 결과에 대해 루프 실행
    for pred_row in preds:
        indices = pred_row.nonzero()[0]
        predicted_labels = mlb.classes_[indices]
        all_predicted_labels.append(predicted_labels.tolist())

    return all_predicted_labels

# 2. 테스트할 문장 리스트 준비
test_sentences = [
"Your shopping cart is empty.",

"Please log in to check your order history.",

"Free shipping on orders over $50.",

"Enter your email to subscribe to our newsletter.",

"Forgot your password?"
]

# 3. 배치 예측 함수 실행
predictions = predict_batch(test_sentences)

# 4. 결과 출력
print("--- 배치 예측 결과 ---")
for text, pred_labels in zip(test_sentences, predictions):
    print(f"\n입력 텍스트: '{text}'")
    print(f"예측된 Predicate: {pred_labels}")

--- 배치 예측 결과 ---

입력 텍스트: 'Your shopping cart is empty.'
예측된 Predicate: ['Not Dark Pattern']

입력 텍스트: 'Please log in to check your order history.'
예측된 Predicate: ['Not Dark Pattern']

입력 텍스트: 'Free shipping on orders over $50.'
예측된 Predicate: ['Pressured Selling']

입력 텍스트: 'Enter your email to subscribe to our newsletter.'
예측된 Predicate: ['Not Dark Pattern']

입력 텍스트: 'Forgot your password?'
예측된 Predicate: ['Not Dark Pattern']


# Activity Notifications이여야 함

In [23]:
# 1. 여러 문장을 한 번에 예측하는 함수 정의
def predict_batch(texts: list):
    model.eval()  # 모델을 평가 모드로 설정

    # ✨ 변경점 1: 토크나이저에 문장 리스트를 그대로 전달하고, padding=True 옵션 추가
    encoding = tokenizer(
        texts,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding=True,  # 배치 내 문장 길이를 맞춰줌
        truncation=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.sigmoid(outputs)

    # ✨ 변경점 2: 배치 전체의 결과를 한 번에 처리
    # 임계값(0.5)을 기준으로 예측 결정
    preds = (probs > 0.5).cpu().numpy()

    all_predicted_labels = []
    # 각 문장의 예측 결과에 대해 루프 실행
    for pred_row in preds:
        indices = pred_row.nonzero()[0]
        predicted_labels = mlb.classes_[indices]
        all_predicted_labels.append(predicted_labels.tolist())

    return all_predicted_labels

# 2. 테스트할 문장 리스트 준비
test_sentences = [
"Someone in Seoul just purchased this item!",

"25 people are viewing this hotel right now.",

"Alex from Busan just added this to their cart.",

"The last one was sold 5 minutes ago.",

"A popular choice for travelers from your area."
]

# 3. 배치 예측 함수 실행
predictions = predict_batch(test_sentences)

# 4. 결과 출력
print("--- 배치 예측 결과 ---")
for text, pred_labels in zip(test_sentences, predictions):
    print(f"\n입력 텍스트: '{text}'")
    print(f"예측된 Predicate: {pred_labels}")

--- 배치 예측 결과 ---

입력 텍스트: 'Someone in Seoul just purchased this item!'
예측된 Predicate: ['Activity Notifications']

입력 텍스트: '25 people are viewing this hotel right now.'
예측된 Predicate: ['Activity Notifications']

입력 텍스트: 'Alex from Busan just added this to their cart.'
예측된 Predicate: ['Activity Notifications']

입력 텍스트: 'The last one was sold 5 minutes ago.'
예측된 Predicate: ['Activity Notifications']

입력 텍스트: 'A popular choice for travelers from your area.'
예측된 Predicate: ['Pressured Selling']


# Countdown Timers이여야 함

In [24]:
# 1. 여러 문장을 한 번에 예측하는 함수 정의
def predict_batch(texts: list):
    model.eval()  # 모델을 평가 모드로 설정

    # ✨ 변경점 1: 토크나이저에 문장 리스트를 그대로 전달하고, padding=True 옵션 추가
    encoding = tokenizer(
        texts,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding=True,  # 배치 내 문장 길이를 맞춰줌
        truncation=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.sigmoid(outputs)

    # ✨ 변경점 2: 배치 전체의 결과를 한 번에 처리
    # 임계값(0.5)을 기준으로 예측 결정
    preds = (probs > 0.5).cpu().numpy()

    all_predicted_labels = []
    # 각 문장의 예측 결과에 대해 루프 실행
    for pred_row in preds:
        indices = pred_row.nonzero()[0]
        predicted_labels = mlb.classes_[indices]
        all_predicted_labels.append(predicted_labels.tolist())

    return all_predicted_labels

# 2. 테스트할 문장 리스트 준비
test_sentences = [
"Special offer ends in 10:00!",

"Today's special! Time left: 02:45:12",

"Hurry! The items in your cart will be removed in 05:00.",

"This price is only available for the next 20 minutes.",

"Flash sale ends in: 00:15:30"
]

# 3. 배치 예측 함수 실행
predictions = predict_batch(test_sentences)

# 4. 결과 출력
print("--- 배치 예측 결과 ---")
for text, pred_labels in zip(test_sentences, predictions):
    print(f"\n입력 텍스트: '{text}'")
    print(f"예측된 Predicate: {pred_labels}")

--- 배치 예측 결과 ---

입력 텍스트: 'Special offer ends in 10:00!'
예측된 Predicate: ['Countdown Timers']

입력 텍스트: 'Today's special! Time left: 02:45:12'
예측된 Predicate: ['Countdown Timers']

입력 텍스트: 'Hurry! The items in your cart will be removed in 05:00.'
예측된 Predicate: ['Countdown Timers']

입력 텍스트: 'This price is only available for the next 20 minutes.'
예측된 Predicate: ['Countdown Timers']

입력 텍스트: 'Flash sale ends in: 00:15:30'
예측된 Predicate: ['Countdown Timers']


# Limited-time Messages이여야 함

In [25]:
# 1. 여러 문장을 한 번에 예측하는 함수 정의
def predict_batch(texts: list):
    model.eval()  # 모델을 평가 모드로 설정

    # ✨ 변경점 1: 토크나이저에 문장 리스트를 그대로 전달하고, padding=True 옵션 추가
    encoding = tokenizer(
        texts,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding=True,  # 배치 내 문장 길이를 맞춰줌
        truncation=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.sigmoid(outputs)

    # ✨ 변경점 2: 배치 전체의 결과를 한 번에 처리
    # 임계값(0.5)을 기준으로 예측 결정
    preds = (probs > 0.5).cpu().numpy()

    all_predicted_labels = []
    # 각 문장의 예측 결과에 대해 루프 실행
    for pred_row in preds:
        indices = pred_row.nonzero()[0]
        predicted_labels = mlb.classes_[indices]
        all_predicted_labels.append(predicted_labels.tolist())

    return all_predicted_labels

# 2. 테스트할 문장 리스트 준비
test_sentences = [
"Today only! 50% off all items.",

"Don't miss out, this weekend only!",

"This exclusive bundle is available for a limited time.",

"24-hour flash sale starts now.",

"Offer ends tonight at midnight."
]

# 3. 배치 예측 함수 실행
predictions = predict_batch(test_sentences)

# 4. 결과 출력
print("--- 배치 예측 결과 ---")
for text, pred_labels in zip(test_sentences, predictions):
    print(f"\n입력 텍스트: '{text}'")
    print(f"예측된 Predicate: {pred_labels}")

--- 배치 예측 결과 ---

입력 텍스트: 'Today only! 50% off all items.'
예측된 Predicate: ['Limited-time Messages']

입력 텍스트: 'Don't miss out, this weekend only!'
예측된 Predicate: ['Limited-time Messages']

입력 텍스트: 'This exclusive bundle is available for a limited time.'
예측된 Predicate: ['Limited-time Messages']

입력 텍스트: '24-hour flash sale starts now.'
예측된 Predicate: ['Countdown Timers']

입력 텍스트: 'Offer ends tonight at midnight.'
예측된 Predicate: ['Limited-time Messages']


# Low-stock Messages이여야 함

In [26]:
# 1. 여러 문장을 한 번에 예측하는 함수 정의
def predict_batch(texts: list):
    model.eval()  # 모델을 평가 모드로 설정

    # ✨ 변경점 1: 토크나이저에 문장 리스트를 그대로 전달하고, padding=True 옵션 추가
    encoding = tokenizer(
        texts,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding=True,  # 배치 내 문장 길이를 맞춰줌
        truncation=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.sigmoid(outputs)

    # ✨ 변경점 2: 배치 전체의 결과를 한 번에 처리
    # 임계값(0.5)을 기준으로 예측 결정
    preds = (probs > 0.5).cpu().numpy()

    all_predicted_labels = []
    # 각 문장의 예측 결과에 대해 루프 실행
    for pred_row in preds:
        indices = pred_row.nonzero()[0]
        predicted_labels = mlb.classes_[indices]
        all_predicted_labels.append(predicted_labels.tolist())

    return all_predicted_labels

# 2. 테스트할 문장 리스트 준비
test_sentences = [
"Hurry! Only 3 left in stock.",

"Almost sold out! Buy now.",

"This is a popular item and stock is running low.",

"Get it before it's gone.",

"Only 1 remaining!"
]

# 3. 배치 예측 함수 실행
predictions = predict_batch(test_sentences)

# 4. 결과 출력
print("--- 배치 예측 결과 ---")
for text, pred_labels in zip(test_sentences, predictions):
    print(f"\n입력 텍스트: '{text}'")
    print(f"예측된 Predicate: {pred_labels}")


--- 배치 예측 결과 ---

입력 텍스트: 'Hurry! Only 3 left in stock.'
예측된 Predicate: ['Low-stock Messages']

입력 텍스트: 'Almost sold out! Buy now.'
예측된 Predicate: ['Not Dark Pattern']

입력 텍스트: 'This is a popular item and stock is running low.'
예측된 Predicate: ['Low-stock Messages']

입력 텍스트: 'Get it before it's gone.'
예측된 Predicate: ['Not Dark Pattern']

입력 텍스트: 'Only 1 remaining!'
예측된 Predicate: ['Limited-time Messages']


# Confirmshaming이여야 함

In [27]:
# 1. 여러 문장을 한 번에 예측하는 함수 정의
def predict_batch(texts: list):
    model.eval()  # 모델을 평가 모드로 설정

    # ✨ 변경점 1: 토크나이저에 문장 리스트를 그대로 전달하고, padding=True 옵션 추가
    encoding = tokenizer(
        texts,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding=True,  # 배치 내 문장 길이를 맞춰줌
        truncation=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.sigmoid(outputs)

    # ✨ 변경점 2: 배치 전체의 결과를 한 번에 처리
    # 임계값(0.5)을 기준으로 예측 결정
    preds = (probs > 0.5).cpu().numpy()

    all_predicted_labels = []
    # 각 문장의 예측 결과에 대해 루프 실행
    for pred_row in preds:
        indices = pred_row.nonzero()[0]
        predicted_labels = mlb.classes_[indices]
        all_predicted_labels.append(predicted_labels.tolist())

    return all_predicted_labels

# 2. 테스트할 문장 리스트 준비
test_sentences = [
"Yes, I want 15% off!", "No thanks, I like paying full price.",

"Yes, keep me in the loop!" ,"No, I'd rather be left behind.",

"Yes, I want free shipping.", "No, I'm happy to pay for shipping.",

"Yes! I want special offers.", "No thanks, I'll pass on the savings.",

"Yes, protect my purchase.", "No thanks, I'll take the risk."
]

# 3. 배치 예측 함수 실행
predictions = predict_batch(test_sentences)

# 4. 결과 출력
print("--- 배치 예측 결과 ---")
for text, pred_labels in zip(test_sentences, predictions):
    print(f"\n입력 텍스트: '{text}'")
    print(f"예측된 Predicate: {pred_labels}")


--- 배치 예측 결과 ---

입력 텍스트: 'Yes, I want 15% off!'
예측된 Predicate: ['Confirmshaming']

입력 텍스트: 'No thanks, I like paying full price.'
예측된 Predicate: ['Confirmshaming']

입력 텍스트: 'Yes, keep me in the loop!'
예측된 Predicate: ['Not Dark Pattern']

입력 텍스트: 'No, I'd rather be left behind.'
예측된 Predicate: ['Confirmshaming']

입력 텍스트: 'Yes, I want free shipping.'
예측된 Predicate: ['Confirmshaming']

입력 텍스트: 'No, I'm happy to pay for shipping.'
예측된 Predicate: ['Confirmshaming']

입력 텍스트: 'Yes! I want special offers.'
예측된 Predicate: ['Confirmshaming']

입력 텍스트: 'No thanks, I'll pass on the savings.'
예측된 Predicate: ['Confirmshaming']

입력 텍스트: 'Yes, protect my purchase.'
예측된 Predicate: ['Pressured Selling']

입력 텍스트: 'No thanks, I'll take the risk.'
예측된 Predicate: ['Confirmshaming']


# Pressured Selling이여야 함

In [29]:
# 1. 여러 문장을 한 번에 예측하는 함수 정의
def predict_batch(texts: list):
    model.eval()  # 모델을 평가 모드로 설정

    # ✨ 변경점 1: 토크나이저에 문장 리스트를 그대로 전달하고, padding=True 옵션 추가
    encoding = tokenizer(
        texts,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding=True,  # 배치 내 문장 길이를 맞춰줌
        truncation=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.sigmoid(outputs)

    # ✨ 변경점 2: 배치 전체의 결과를 한 번에 처리
    # 임계값(0.5)을 기준으로 예측 결정
    preds = (probs > 0.5).cpu().numpy()

    all_predicted_labels = []
    # 각 문장의 예측 결과에 대해 루프 실행
    for pred_row in preds:
        indices = pred_row.nonzero()[0]
        predicted_labels = mlb.classes_[indices]
        all_predicted_labels.append(predicted_labels.tolist())

    return all_predicted_labels

# 2. 테스트할 문장 리스트 준비
test_sentences = [
"Someone else might buy this if you don't act now!",

"This item is in many other carts. Complete your purchase before it's gone.",

"Last chance! The price will increase soon.",

"Your items are not reserved. Check out now to avoid disappointment.",

"If you wait, it'll be sold out!"
]

# 3. 배치 예측 함수 실행
predictions = predict_batch(test_sentences)

# 4. 결과 출력
print("--- 배치 예측 결과 ---")
for text, pred_labels in zip(test_sentences, predictions):
    print(f"\n입력 텍스트: '{text}'")
    print(f"예측된 Predicate: {pred_labels}")


--- 배치 예측 결과 ---

입력 텍스트: 'Someone else might buy this if you don't act now!'
예측된 Predicate: []

입력 텍스트: 'This item is in many other carts. Complete your purchase before it's gone.'
예측된 Predicate: []

입력 텍스트: 'Last chance! The price will increase soon.'
예측된 Predicate: ['Limited-time Messages']

입력 텍스트: 'Your items are not reserved. Check out now to avoid disappointment.'
예측된 Predicate: ['Limited-time Messages']

입력 텍스트: 'If you wait, it'll be sold out!'
예측된 Predicate: ['Not Dark Pattern']


# High-demand Messages이여야 함

In [30]:
# 1. 여러 문장을 한 번에 예측하는 함수 정의
def predict_batch(texts: list):
    model.eval()  # 모델을 평가 모드로 설정

    # ✨ 변경점 1: 토크나이저에 문장 리스트를 그대로 전달하고, padding=True 옵션 추가
    encoding = tokenizer(
        texts,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding=True,  # 배치 내 문장 길이를 맞춰줌
        truncation=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.sigmoid(outputs)

    # ✨ 변경점 2: 배치 전체의 결과를 한 번에 처리
    # 임계값(0.5)을 기준으로 예측 결정
    preds = (probs > 0.5).cpu().numpy()

    all_predicted_labels = []
    # 각 문장의 예측 결과에 대해 루프 실행
    for pred_row in preds:
        indices = pred_row.nonzero()[0]
        predicted_labels = mlb.classes_[indices]
        all_predicted_labels.append(predicted_labels.tolist())

    return all_predicted_labels

# 2. 테스트할 문장 리스트 준비
test_sentences = [
"Over 1,000 sold in the last 24 hours!",

"Our hottest item right now!",

"This product is selling fast.",

"57 people have bought this in the last hour.",

"As seen on TV!"
]

# 3. 배치 예측 함수 실행
predictions = predict_batch(test_sentences)

# 4. 결과 출력
print("--- 배치 예측 결과 ---")
for text, pred_labels in zip(test_sentences, predictions):
    print(f"\n입력 텍스트: '{text}'")
    print(f"예측된 Predicate: {pred_labels}")


--- 배치 예측 결과 ---

입력 텍스트: 'Over 1,000 sold in the last 24 hours!'
예측된 Predicate: ['Activity Notifications']

입력 텍스트: 'Our hottest item right now!'
예측된 Predicate: ['Activity Notifications']

입력 텍스트: 'This product is selling fast.'
예측된 Predicate: ['High-demand Messages']

입력 텍스트: '57 people have bought this in the last hour.'
예측된 Predicate: ['Activity Notifications']

입력 텍스트: 'As seen on TV!'
예측된 Predicate: ['Not Dark Pattern']


# Trick Questions이여야 함

In [31]:
# 1. 여러 문장을 한 번에 예측하는 함수 정의
def predict_batch(texts: list):
    model.eval()  # 모델을 평가 모드로 설정

    # ✨ 변경점 1: 토크나이저에 문장 리스트를 그대로 전달하고, padding=True 옵션 추가
    encoding = tokenizer(
        texts,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding=True,  # 배치 내 문장 길이를 맞춰줌
        truncation=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.sigmoid(outputs)

    # ✨ 변경점 2: 배치 전체의 결과를 한 번에 처리
    # 임계값(0.5)을 기준으로 예측 결정
    preds = (probs > 0.5).cpu().numpy()

    all_predicted_labels = []
    # 각 문장의 예측 결과에 대해 루프 실행
    for pred_row in preds:
        indices = pred_row.nonzero()[0]
        predicted_labels = mlb.classes_[indices]
        all_predicted_labels.append(predicted_labels.tolist())

    return all_predicted_labels

# 2. 테스트할 문장 리스트 준비
test_sentences = [
"[ ] Uncheck this box if you DO NOT want to stop receiving marketing emails.",

"Do you disagree with the terms below? (O) Yes (O) No",

"Yes, sign me up for the $9.99/month premium service.",

"By clicking below, you agree to withdraw your refusal to receive promotional content.",

"To proceed, you must subscribe to all optional add-on services."
]

# 3. 배치 예측 함수 실행
predictions = predict_batch(test_sentences)

# 4. 결과 출력
print("--- 배치 예측 결과 ---")
for text, pred_labels in zip(test_sentences, predictions):
    print(f"\n입력 텍스트: '{text}'")
    print(f"예측된 Predicate: {pred_labels}")


--- 배치 예측 결과 ---

입력 텍스트: '[ ] Uncheck this box if you DO NOT want to stop receiving marketing emails.'
예측된 Predicate: ['Trick Questions']

입력 텍스트: 'Do you disagree with the terms below? (O) Yes (O) No'
예측된 Predicate: ['Not Dark Pattern']

입력 텍스트: 'Yes, sign me up for the $9.99/month premium service.'
예측된 Predicate: ['Pressured Selling']

입력 텍스트: 'By clicking below, you agree to withdraw your refusal to receive promotional content.'
예측된 Predicate: ['Trick Questions']

입력 텍스트: 'To proceed, you must subscribe to all optional add-on services.'
예측된 Predicate: []


# Testimonials of Uncertain Origin이여야 함

In [32]:
# 1. 여러 문장을 한 번에 예측하는 함수 정의
def predict_batch(texts: list):
    model.eval()  # 모델을 평가 모드로 설정

    # ✨ 변경점 1: 토크나이저에 문장 리스트를 그대로 전달하고, padding=True 옵션 추가
    encoding = tokenizer(
        texts,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding=True,  # 배치 내 문장 길이를 맞춰줌
        truncation=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.sigmoid(outputs)

    # ✨ 변경점 2: 배치 전체의 결과를 한 번에 처리
    # 임계값(0.5)을 기준으로 예측 결정
    preds = (probs > 0.5).cpu().numpy()

    all_predicted_labels = []
    # 각 문장의 예측 결과에 대해 루프 실행
    for pred_row in preds:
        indices = pred_row.nonzero()[0]
        predicted_labels = mlb.classes_[indices]
        all_predicted_labels.append(predicted_labels.tolist())

    return all_predicted_labels

# 2. 테스트할 문장 리스트 준비
test_sentences = [
"This is the best product I've ever used! - Jane D.",

"I lost 10 lbs in one week with this, it really works! - A happy customer.",

"Five stars! Highly recommended. - John S.",

"It changed my life! Don't hesitate. - User123",

"Fast shipping and a perfect product. Thanks."
]

# 3. 배치 예측 함수 실행
predictions = predict_batch(test_sentences)

# 4. 결과 출력
print("--- 배치 예측 결과 ---")
for text, pred_labels in zip(test_sentences, predictions):
    print(f"\n입력 텍스트: '{text}'")
    print(f"예측된 Predicate: {pred_labels}")


--- 배치 예측 결과 ---

입력 텍스트: 'This is the best product I've ever used! - Jane D.'
예측된 Predicate: ['Not Dark Pattern']

입력 텍스트: 'I lost 10 lbs in one week with this, it really works! - A happy customer.'
예측된 Predicate: ['Activity Notifications']

입력 텍스트: 'Five stars! Highly recommended. - John S.'
예측된 Predicate: ['Pressured Selling']

입력 텍스트: 'It changed my life! Don't hesitate. - User123'
예측된 Predicate: []

입력 텍스트: 'Fast shipping and a perfect product. Thanks.'
예측된 Predicate: ['Not Dark Pattern']
